In [2]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    raise ValueError("DATABASE_URL not found in .env file")

engine = create_engine(DATABASE_URL)

def get_engine():
    return engine

## Load Data

In [7]:
def load_data():
    orders = pd.read_sql("SELECT * FROM orders", engine)
    order_items = pd.read_sql("SELECT * FROM order_items", engine)
    products = pd.read_sql("SELECT * FROM products", engine)
    return orders, order_items, products

orders, order_items, products = load_data()
print("Data loaded successfully.")
print(f"Orders: {len(orders)}, Order Items: {len(order_items)}, Products: {len(products)}")

Data loaded successfully.
Orders: 67, Order Items: 43, Products: 83


## Outlier Detection (IQR)

In [8]:
def detect_outliers(series: pd.Series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return series[(series < lower_bound) | (series > upper_bound)]

quantity_outliers = detect_outliers(order_items["quantity"])
price_outliers = detect_outliers(products["price"])
total_outliers = detect_outliers(orders["total_amount"])

print(f"Quantity outliers: {len(quantity_outliers)}")
print(f"Price outliers: {len(price_outliers)}")
print(f"Order total outliers: {len(total_outliers)}")

Quantity outliers: 3
Price outliers: 15
Order total outliers: 11


## Date Validation

In [9]:
def validate_dates(df: pd.DataFrame, date_column: str):
    df[date_column] = pd.to_datetime(df[date_column], errors="coerce")
    future_dates = df[df[date_column] > pd.Timestamp.now()]
    invalid_dates = df[df[date_column].isnull()]
    return future_dates, invalid_dates

future_orders, invalid_order_dates = validate_dates(orders, "created_at")
print(f"Orders with future dates: {len(future_orders)}")
print(f"Orders with invalid dates: {len(invalid_order_dates)}")

Orders with future dates: 4
Orders with invalid dates: 0


## Integrity Checks

In [10]:
def check_integrity(order_items: pd.DataFrame, products: pd.DataFrame):
    invalid_products = order_items[
        ~order_items["product_id"].isin(products["product_id"])
    ]
    return invalid_products

invalid_product_refs = check_integrity(order_items, products)
orders_without_items = orders[
    ~orders["order_id"].isin(order_items["order_id"])
]

print(f"Order items with non-existing products: {len(invalid_product_refs)}")
print(f"Orders without items: {len(orders_without_items)}")

Order items with non-existing products: 0
Orders without items: 24


## Generate Report

In [11]:
report = []
report.append("DATA CLEANING REPORT")
report.append("=" * 50)
report.append("\nOUTLIER ANALYSIS")
report.append(f"Quantity outliers detected: {len(quantity_outliers)}")
report.append(f"Price outliers detected: {len(price_outliers)}")
report.append(f"Order total outliers detected: {len(total_outliers)}")
report.append("\nDATE VALIDATION")
report.append(f"Orders with future dates: {len(future_orders)}")
report.append(f"Orders with invalid dates: {len(invalid_order_dates)}")
report.append("\nINTEGRITY CHECKS")
report.append(f"Order items with non-existing products: {len(invalid_product_refs)}")
report.append(f"Orders without items: {len(orders_without_items)}")

with open("../reports/data_cleaning_report.txt", "w") as file:
    file.write("\n".join(report))

print("Data cleaning report generated successfully.")
print("\n".join(report))

Data cleaning report generated successfully.
DATA CLEANING REPORT

OUTLIER ANALYSIS
Quantity outliers detected: 3
Price outliers detected: 15
Order total outliers detected: 11

DATE VALIDATION
Orders with future dates: 4
Orders with invalid dates: 0

INTEGRITY CHECKS
Order items with non-existing products: 0
Orders without items: 24
